In [103]:
import numpy as np
import pandas as pd

In [104]:
df = pd.read_json('purchase_data.json')
#df = pd.read_json('purchase_data2.json')

In [105]:
#getting sense of data 
df.head()

,Age,Gender,Item ID,Item Name,Price,SN
0,20,Male,93,Apocalyptic Battlescythe,4.49,Iloni35
1,21,Male,12,Dawne,3.36,Aidaira26
2,17,Male,5,Putrid Fan,2.63,Irim47
3,17,Male,123,Twilight's Carver,2.55,Irith83
4,22,Male,154,Feral Katana,4.11,Philodil43


In [106]:
# Preparing data : Changing name of Gender column to Gender Demographics

df = df.rename(columns={'Gender' : 'Gender Demographics' })

df.head()

,Age,Gender Demographics,Item ID,Item Name,Price,SN
0,20,Male,93,Apocalyptic Battlescythe,4.49,Iloni35
1,21,Male,12,Dawne,3.36,Aidaira26
2,17,Male,5,Putrid Fan,2.63,Irim47
3,17,Male,123,Twilight's Carver,2.55,Irith83
4,22,Male,154,Feral Katana,4.11,Philodil43


In [107]:
# Preparing data continue --Adding Age Demographics group 

my_source_series = df['Age']
subtract_lable_display=True
bin_precison = 5
upper_bin = int(bin_precison*np.ceil(my_source_series.max()/bin_precison)) 
lower_bin = int(bin_precison*np.floor(my_source_series.min()/bin_precison)) 


myList = []
i = lower_bin
while i <= upper_bin:
    myList.append(i)
    i += bin_precison 


myLabelList = []
myLabelList.append(myList[0])

if subtract_lable_display==True:
    upper_subtract =1
else :
    upper_subtract = 0


j = 1
while j < len(myList):
    mylabel = str(myList[j-1]) + '-' +str( myList[j]-upper_subtract)
    myLabelList.append(mylabel)
    j += 1
myLabelList.pop(0)



#------custom upper bound and lower bound
lowest_list = myLabelList[0].split('-')
myLabelList[0] = '<' + str(int(lowest_list[1])+upper_subtract)

highest_list = myLabelList[len(myLabelList)-1].split('-')
myLabelList[len(myLabelList)-1] = highest_list[0] + '+'
#----------------------------------


df['Age Demographics'] = pd.cut(my_source_series,myList, labels = myLabelList,precision=0,include_lowest=True, right= False)

# upper limit correction
df.loc[(df['Age'] == upper_bin), ['Age Demographics'] ] = myLabelList[len(myLabelList)-1]


df.head()

,Age,Gender Demographics,Item ID,Item Name,Price,SN,Age Demographics
0,20,Male,93,Apocalyptic Battlescythe,4.49,Iloni35,20-24
1,21,Male,12,Dawne,3.36,Aidaira26,20-24
2,17,Male,5,Putrid Fan,2.63,Irim47,15-19
3,17,Male,123,Twilight's Carver,2.55,Irith83,15-19
4,22,Male,154,Feral Katana,4.11,Philodil43,20-24


In [108]:
#create seperate personal data frame ---normalizing data 
result_list = ['SN','Age','Gender Demographics','Age Demographics']
df_personal = df.loc[:,result_list].drop_duplicates()
df_personal.head()

,SN,Age,Gender Demographics,Age Demographics
0,Iloni35,20,Male,20-24
1,Aidaira26,21,Male,20-24
2,Irim47,17,Male,15-19
3,Irith83,17,Male,15-19
4,Philodil43,22,Male,20-24


In [109]:
#Finding total counts of players #Result

total_players = df_personal['SN'].count()
Result_PlayerCount = pd.DataFrame([{'Total Players' : total_players}])         
Result_PlayerCount

,Total Players
0,74


In [110]:
# Run Aggregation on purchasing data 

agg_dict = { 'Item ID': ['nunique', 'count'] ,'Price': ['mean', 'sum' ] }
df_PurchasingAnalysis = df.agg(agg_dict)

df_PurchasingAnalysis

,Item ID,Price
count,78.0,NaN
mean,NaN,2.924359
nunique,64.0,NaN
sum,NaN,228.100000


In [111]:
# Purchasing Analysis (Total) #result

d1 =  {'Number of Unique Items' : df_PurchasingAnalysis.iloc[2,0 ],  
       'Average Price' : df_PurchasingAnalysis.iloc[1,1 ], 
       'Number of Purchases' : df_PurchasingAnalysis.iloc[0,0 ] , 
       'Total Revenue' : df_PurchasingAnalysis.iloc[3,1 ]
      }

Result_PurchasingAnalysis= pd.DataFrame([d1])

Result_PurchasingAnalysis

#TODO format



,Average Price,Number of Purchases,Number of Unique Items,Total Revenue
0,2.924359,78.0,64.0,228.1


In [112]:
# Gender Demographics #result

agg_dict = { 'SN': ['count'] }
groupbytype_GenderDemographics = df_personal.groupby(['Gender Demographics'])
Result_GenderDemographics = groupbytype_GenderDemographics.agg(agg_dict)
Result_GenderDemographics.columns = ["_".join(x) for x in Result_GenderDemographics.columns.ravel()]

Result_GenderDemographics = Result_GenderDemographics.rename(columns={'SN_count' : 'Percentage of Players' })
Result_GenderDemographics['Total Count'] = Result_GenderDemographics['Percentage of Players']
Result_GenderDemographics['Percentage of Players'] = round(100*Result_GenderDemographics['Percentage of Players']/total_players,2)

Result_GenderDemographics

,Percentage of Players,Total Count
Gender Demographics,,
Female,17.57,13
Male,81.08,60
Other / Non-Disclosed,1.35,1


In [113]:
# Purchasing Analysis (Gender) #result
agg_dict = { 'SN': ['nunique'],'Price': ['mean', 'sum' ,'count']  }
groupbytype_GenderDemographics1 = df.groupby(['Gender Demographics'])
Result_GenderDemographics1 = groupbytype_GenderDemographics1.agg(agg_dict)
Result_GenderDemographics1.columns = ["_".join(x) for x in Result_GenderDemographics1.columns.ravel()]
Result_GenderDemographics1 = Result_GenderDemographics1.rename(columns={'SN_nunique' : 'Number of Players' , 'Price_count' : 'Purchase Count','Price_sum' : 'Total Purchase Value','Price_mean' : 'Average Purchase Price' })
Result_GenderDemographics1['Normalized Total'] = round(Result_GenderDemographics1['Total Purchase Value']/Result_GenderDemographics1['Number of Players'],2)

Result_GenderDemographics1.drop('Number of Players', axis=1 ,  inplace=True)

Result_GenderDemographics1

#TODO format



,Average Purchase Price,Total Purchase Value,Purchase Count,Normalized Total
Gender Demographics,,,,
Female,3.183077,41.38,13,3.18
Male,2.884375,184.60,64,3.08
Other / Non-Disclosed,2.120000,2.12,1,2.12


In [114]:
# Age Demographics #result

agg_dict = { 'SN': ['count'] }
groupbytype_AgeDemographics = df_personal.groupby(['Age Demographics'])
Result_AgeDemographics = groupbytype_AgeDemographics.agg(agg_dict)
Result_AgeDemographics.columns = ["_".join(x) for x in Result_AgeDemographics.columns.ravel()]

Result_AgeDemographics = Result_AgeDemographics.rename(columns={'SN_count' : 'Percentage of Players' })
Result_AgeDemographics['Total Count'] = Result_AgeDemographics['Percentage of Players']
Result_AgeDemographics['Percentage of Players'] = round(100*Result_AgeDemographics['Percentage of Players']/total_players,2)

Result_AgeDemographics

,Percentage of Players,Total Count
Age Demographics,,
<10,6.76,5
10-14,4.05,3
15-19,14.86,11
20-24,45.95,34
25-29,10.81,8
30-34,8.11,6
35+,9.46,7


In [115]:
# Purchasing Analysis (Age) #result
agg_dict = { 'SN': ['nunique'],'Price': ['mean', 'sum' ,'count']  }
groupbytype_AgeDemographics1 = df.groupby(['Age Demographics'])
Result_AgeDemographics1 = groupbytype_AgeDemographics1.agg(agg_dict)
Result_AgeDemographics1.columns = ["_".join(x) for x in Result_AgeDemographics1.columns.ravel()]
Result_AgeDemographics1 = Result_AgeDemographics1.rename(columns={'SN_nunique' : 'Number of Players' , 'Price_count' : 'Purchase Count','Price_sum' : 'Total Purchase Value','Price_mean' : 'Average Purchase Price' })
Result_AgeDemographics1['Normalized Total'] = round(Result_AgeDemographics1['Total Purchase Value']/Result_AgeDemographics1['Number of Players'],2)

Result_AgeDemographics1.drop('Number of Players', axis=1 ,  inplace=True)

Result_AgeDemographics1

#TODO format




,Average Purchase Price,Total Purchase Value,Purchase Count,Normalized Total
Age Demographics,,,,
<10,2.764000,13.82,5,2.76
10-14,2.986667,8.96,3,2.99
15-19,2.764545,30.41,11,2.76
20-24,3.024722,108.89,36,3.20
25-29,2.901111,26.11,9,3.26
30-34,1.984286,13.89,7,2.32
35+,3.717143,26.02,7,3.72


In [116]:
# Top Spenders (SN) #result
agg_dict = { 'SN': ['nunique'],'Price': ['mean', 'sum' ,'count']  }
groupbytype_SNDemographics1 = df.groupby(['SN'])
Result_SNDemographics1 = groupbytype_SNDemographics1.agg(agg_dict)
Result_SNDemographics1.columns = ["_".join(x) for x in Result_SNDemographics1.columns.ravel()]
Result_SNDemographics1 = Result_SNDemographics1.rename(columns={'SN_nunique' : 'Number of Players' , 'Price_count' : 'Purchase Count','Price_sum' : 'Total Purchase Value','Price_mean' : 'Average Purchase Price' })

Result_SNDemographics1.drop('Number of Players', axis=1 ,  inplace=True)

Result_SNDemographicssorted=Result_SNDemographics1.sort_values(['Total Purchase Value'], ascending= False).head(5)
Result_SNDemographicssorted

#TODO format



,Average Purchase Price,Total Purchase Value,Purchase Count
SN,,,
Sundaky74,3.705,7.41,2
Aidaira26,2.565,5.13,2
Eusty71,4.810,4.81,1
Chanirra64,4.780,4.78,1
Alarap40,4.710,4.71,1


In [117]:
# Most Popular Items #result

agg_dict = {'Price': ['max', 'sum' ,'count']    }
groupbytype_ItemDemographics1= df.groupby(['Item ID','Item Name'])
Result_ItemDemographics1= groupbytype_ItemDemographics1.agg(agg_dict) 
Result_ItemDemographics1.columns = ["_".join(x) for x in Result_ItemDemographics1.columns.ravel()]

Result_ItemDemographics1 = Result_ItemDemographics1.rename(columns={'Price_count' : 'Purchase Count','Price_sum' : 'Total Purchase Value','Price_max' : 'Item Price' })
Result_ItemDemographics1['Total Purchase Value'] = Result_ItemDemographics1['Purchase Count'] * Result_ItemDemographics1['Item Price']

Result_ItemDemographicssorted=Result_ItemDemographics1.sort_values(['Purchase Count'], ascending= False).head(5)
Result_ItemDemographicssorted

,,Item Price,Total Purchase Value,Purchase Count
Item ID,Item Name,,,
94,Mourning Blade,3.64,10.92,3
90,Betrayer,4.12,8.24,2
111,Misery's End,1.79,3.58,2
64,Fusion Pummel,2.42,4.84,2
154,Feral Katana,4.11,8.22,2


In [118]:
# Most Profitable Items #result

Result_ItemDemographicssorted2=Result_ItemDemographics1.sort_values(['Total Purchase Value'], ascending= False).head(5)
Result_ItemDemographicssorted2

,,Item Price,Total Purchase Value,Purchase Count
Item ID,Item Name,,,
94,Mourning Blade,3.64,10.92,3
117,"Heartstriker, Legacy of the Light",4.71,9.42,2
93,Apocalyptic Battlescythe,4.49,8.98,2
90,Betrayer,4.12,8.24,2
154,Feral Katana,4.11,8.22,2
